In [1]:
!pip -q install torch_geometric rectools
!pip -q install comet_ml
!pip -q install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 104.2 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 20.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 80.1 MB/s eta 0:00:00:00:01


In [2]:
import comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

from dotenv import load_dotenv
import os

In [ ]:
load_dotenv(".env")

True

In [4]:
experiment = Experiment(
  api_key=os.getenv('API_KEY'),
  project_name="gnn-recommender",
  workspace="annanet",
  log_code=True
)

experiment.set_name('emoSAGE+THP-books')
experiment.add_tags(['books', 'leave-n-out'])

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/annanet/gnn-recommender/5321f7ecb6044c33a138b28b232b1381

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


https://www.comet.com/annanet/gnn-recommender/5321f7ecb6044c33a138b28b232b1381

In [5]:
hyperparameters = {
    'seed': 42,
    'types_of_feedback': ["explicit_positive", "expliсit_negative",
                          "implicit_positive", "implicit_negative"],
    'max_len_of_thp_history': 100,
    'pad_id': 0,         
    'cls_id': None,  # filled in at the stage of creating a story for thp
    'thp_dmodel': 64,  # размер эмбеддингов
    'thp_n_head': 4,  # число attention-голов
    'thp_window_size': 101,  # окно THP
    'thp_decay': 1.0,  # скорость экспоненциального затухания
    'thp_dropout': 0.2,  # dropout
    'train_edge_type': [('item','to_feedback_explicit_positive','explicit_positive'), 
                        ('item','to_feedback_implicit_positive','implicit_positive')],
    'train_num_epochs': 300,
    'train_lr': 1e-3,
    'train_batch_size': 4096,
    'train_print_every': 20,  # раз в сколько шагов печатаем статистику
    'train_test_every': 50,
    'test_topk': 10,
    'test_batch_size': 8192,
    'train_scheduler_step_size': 150,
    'train_scheduler_gamma': 0.98
}

In [6]:
os.listdir('/kaggle/input/data/leave-n-out/books')

['train.csv', 'test.csv']

In [7]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, SAGEConv

from sklearn.preprocessing import LabelEncoder

from rectools import Columns
from rectools.metrics import MAP, Precision, Recall, NDCG, calc_metrics

import gc
import random

In [8]:
SEED = hyperparameters['seed']
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [9]:
rootpath = '/kaggle/input/data/leave-n-out/books/'
train = pd.read_csv(
    rootpath+'train.csv'
)
train['date'] = pd.to_datetime(train['date_added'])
print(train.head())

                            user_id  book_id  rating  \
0  000883382802f2d95a3dd545bb953882  8525590       3   
1  000883382802f2d95a3dd545bb953882  2767052       5   
2  000883382802f2d95a3dd545bb953882  3236307       5   
3  000883382802f2d95a3dd545bb953882   256683       4   
4  000883382802f2d95a3dd545bb953882  6001758       5   

                  date_added                       date  
0  2011-10-31 18:37:56-07:00  2011-10-31 18:37:56-07:00  
1  2011-10-31 18:40:33-07:00  2011-10-31 18:40:33-07:00  
2  2011-11-02 08:30:30-07:00  2011-11-02 08:30:30-07:00  
3  2011-11-02 08:55:16-07:00  2011-11-02 08:55:16-07:00  
4  2011-11-02 08:57:21-07:00  2011-11-02 08:57:21-07:00  


/tmp/ipykernel_35/4238775169.py:5: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  train['date'] = pd.to_datetime(train['date_added'])


In [10]:
explicit_positive = train[(train["rating"] == 5)].index
explisit_negative = train[(train["rating"] <= 2)].index

explicit_combined_feedback = explicit_positive.union(explisit_negative)
print('Количество explicit позитивного фидбека', explicit_positive.shape[0])
print('Количество explicit негативного фидбека', explisit_negative.shape[0])

Количество explicit позитивного фидбека 359463
Количество explicit негативного фидбека 178217


In [11]:
implicit_positive = train[(train["rating"] == 4)].index
implicit_negative = train[(train["rating"] == 3)].index

implicit_combined_feedback = implicit_positive.union(implicit_negative)
print('Количество implicit позитивного фидбека', implicit_positive.shape[0])
print('Количество implicit негативного фидбека', implicit_negative.shape[0])

Количество implicit позитивного фидбека 430342
Количество implicit негативного фидбека 258651


In [12]:
train.loc[:, "target"] = ""
train.loc[explicit_positive, "target"] = "explicit_positive"
train.loc[explisit_negative, "target"] = "expliсit_negative"
train.loc[implicit_positive, "target"] = "implicit_positive"
train.loc[implicit_negative, "target"] = "implicit_negative"

train = train[['user_id','book_id','target','date']]
train.head()

,user_id,book_id,target,date
0,000883382802f2d95a3dd545bb953882,8525590,implicit_negative,2011-10-31 18:37:56-07:00
1,000883382802f2d95a3dd545bb953882,2767052,explicit_positive,2011-10-31 18:40:33-07:00
2,000883382802f2d95a3dd545bb953882,3236307,explicit_positive,2011-11-02 08:30:30-07:00
3,000883382802f2d95a3dd545bb953882,256683,implicit_positive,2011-11-02 08:55:16-07:00
4,000883382802f2d95a3dd545bb953882,6001758,explicit_positive,2011-11-02 08:57:21-07:00


In [13]:
train = train.sort_values(by=["user_id", "date"]).reset_index(drop=True)

In [14]:
train

,user_id,book_id,target,date
0,000883382802f2d95a3dd545bb953882,8525590,implicit_negative,2011-10-31 18:37:56-07:00
1,000883382802f2d95a3dd545bb953882,2767052,explicit_positive,2011-10-31 18:40:33-07:00
2,000883382802f2d95a3dd545bb953882,3236307,explicit_positive,2011-11-02 08:30:30-07:00
3,000883382802f2d95a3dd545bb953882,256683,implicit_positive,2011-11-02 08:55:16-07:00
4,000883382802f2d95a3dd545bb953882,6001758,explicit_positive,2011-11-02 08:57:21-07:00
...,...,...,...,...
1226668,ffff7cafdaf5196383cb2efca08fb6fe,17131869,implicit_negative,2015-08-26 16:40:00-07:00
1226669,ffff7cafdaf5196383cb2efca08fb6fe,19358975,implicit_positive,2015-08-26 16:40:01-07:00
1226670,ffff7cafdaf5196383cb2efca08fb6fe,20578940,implicit_positive,2015-08-28 10:23:50-07:00
1226671,ffff7cafdaf5196383cb2efca08fb6fe,18630542,implicit_positive,2015-08-31 18:03:52-07:00


In [15]:
train.columns = ['user_id', 'item_id', 'target', 'date']

In [16]:
test = pd.read_csv(
    rootpath+'test.csv'
)
test['date'] = pd.to_datetime(test['date_added'])
print(test.head())

                            user_id   book_id  rating  \
0  000883382802f2d95a3dd545bb953882   8135807       4   
1  000883382802f2d95a3dd545bb953882  18301124       5   
2  000883382802f2d95a3dd545bb953882  18220354       4   
3  000883382802f2d95a3dd545bb953882  17383918       5   
4  000883382802f2d95a3dd545bb953882  13188676       5   

                  date_added                       date  
0  2013-08-13 09:37:39-07:00  2013-08-13 09:37:39-07:00  
1  2013-10-27 22:18:01-07:00  2013-10-27 22:18:01-07:00  
2  2013-12-09 22:20:59-08:00  2013-12-09 22:20:59-08:00  
3  2013-12-22 20:57:14-08:00  2013-12-22 20:57:14-08:00  
4  2013-12-22 20:58:15-08:00  2013-12-22 20:58:15-08:00  


/tmp/ipykernel_35/2717431719.py:4: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  test['date'] = pd.to_datetime(test['date_added'])


In [17]:
test = test[['user_id','book_id', 'date']]
test.head()

,user_id,book_id,date
0,000883382802f2d95a3dd545bb953882,8135807,2013-08-13 09:37:39-07:00
1,000883382802f2d95a3dd545bb953882,18301124,2013-10-27 22:18:01-07:00
2,000883382802f2d95a3dd545bb953882,18220354,2013-12-09 22:20:59-08:00
3,000883382802f2d95a3dd545bb953882,17383918,2013-12-22 20:57:14-08:00
4,000883382802f2d95a3dd545bb953882,13188676,2013-12-22 20:58:15-08:00


In [18]:
test.columns = ['user_id', 'item_id', 'date']

# MVP model v2

In [19]:
train.loc[:, "event"] = 0
train.loc[(train["target"] == "explicit_positive") | (train["target"] == "implicit_positive"), "event"] = 1

In [20]:
test = test[(test.user_id.isin(train.user_id)) & (test.item_id.isin(train.item_id))].copy()
test.shape

(151126, 3)

In [21]:
# 2. Преобразование данных - для куарека не особо нужно, но для других - напоминалка
# делаем всегда! чтобы не сломать ничего дальше и чтобы все индексы были от 0 до N без пропусков
user_encoder = LabelEncoder()
video_encoder = LabelEncoder()

train.loc[:, 'user_id'] = user_encoder.fit_transform(train['user_id'])
train.loc[:, 'item_id'] = video_encoder.fit_transform(train['item_id'])

test.loc[:, 'user_id'] = user_encoder.transform(test['user_id'])
test.loc[:, 'item_id'] = video_encoder.transform(test['item_id'])

train['user_id'] = train['user_id'].astype(int)
train['item_id'] = train['item_id'].astype(int)
test['user_id'] = test['user_id'].astype(int)
test['item_id'] = test['item_id'].astype(int)

In [22]:
# т.е. сразу знаем количество и в каких пределах изменяется user_id и video_id
num_videos = train['item_id'].nunique()
num_users = train['user_id'].nunique()

print('Количество уникальных item_id', num_videos)
print('Количество уникальных user_id', num_users)

Количество уникальных item_id 25456
Количество уникальных user_id 18892


In [23]:
# так как используем pad, то нумерацию item_id начинаем с 1 до max + 1, чтобы для pad забить 0
train.loc[:, 'item_id'] += 1
test.loc[:, 'item_id'] += 1

In [24]:
import pandas as pd
import torch
from torch_geometric.data import HeteroData


def prepare_hetero_data(df: pd.DataFrame) -> HeteroData:
    """
    Build a heterogeneous graph dataset from interaction records.

    This function constructs a PyTorch Geometric HeteroData object with three types of nodes:
      - 'user': representing each unique user in the dataset
      - 'item': representing each unique item (formerly movie)
      - one node set per feedback type (target), representing feedback interaction events

    Edges are created as follows:
      1) item -> feedback: connecting items to feedback events of type ft
      2) feedback -> item: reverse link from feedback events back to items (for message passing)
      3) feedback -> user: linking each feedback event of type ft to its corresponding user (one-to-one)
      4) user -> user: a complete graph among all users under the relation 'interacts'

    Parameters:
    -----------
    df : pandas.DataFrame
        Input DataFrame must contain the following columns:
          - 'user_id' : integer identifiers for users (0-indexed or otherwise)
          - 'item_id' : integer identifiers for items (e.g. movies), values must be in [0, max(item_id)]
          - 'target' : categorical or numeric label indicating feedback type (e.g. click, purchase)

    Returns:
    --------
    data : torch_geometric.data.HeteroData
        A heterogeneous graph with node types 'user', 'item', and one per feedback label, and edge_index
        tensors appropriately set for message passing in a GNN.
    """
    # Determine the number of users and items
    num_users = df['user_id'].nunique()
    num_items = int(df['item_id'].max()) + 1
    feedback_types = df['target'].unique().tolist()

    # Initialize HeteroData
    data = HeteroData()
    data['user'].node_id = torch.arange(num_users)
    data['item'].node_id = torch.arange(num_items)
    for ft in feedback_types:
        data[ft].node_id = torch.arange(num_users)

    # Build edges: item <-> feedback <-> user
    for ft in feedback_types:
        mask = df['target'] == ft
        items = torch.LongTensor(df.loc[mask, 'item_id'].values)
        users_idx = torch.LongTensor(df.loc[mask, 'user_id'].values)

        # item -> feedback
        data['item', f'to_feedback_{ft}', ft].edge_index = torch.stack([items, users_idx], dim=0)
        # feedback -> item
        data[ft, f'feedback_to_item_{ft}', 'item'].edge_index = torch.stack([users_idx, items], dim=0)
        # feedback -> user (1:1 mapping)
        idx = torch.arange(num_users)
        data[ft, f'to_user_{ft}', 'user'].edge_index = torch.stack([idx, idx], dim=0)

    return data

In [25]:
data = prepare_hetero_data(train)
data

HeteroData(
  user={ node_id=[18892] },
  item={ node_id=[25457] },
  implicit_negative={ node_id=[18892] },
  explicit_positive={ node_id=[18892] },
  implicit_positive={ node_id=[18892] },
  expliсit_negative={ node_id=[18892] },
  (item, to_feedback_implicit_negative, implicit_negative)={ edge_index=[2, 258651] },
  (implicit_negative, feedback_to_item_implicit_negative, item)={ edge_index=[2, 258651] },
  (implicit_negative, to_user_implicit_negative, user)={ edge_index=[2, 18892] },
  (item, to_feedback_explicit_positive, explicit_positive)={ edge_index=[2, 359463] },
  (explicit_positive, feedback_to_item_explicit_positive, item)={ edge_index=[2, 359463] },
  (explicit_positive, to_user_explicit_positive, user)={ edge_index=[2, 18892] },
  (item, to_feedback_implicit_positive, implicit_positive)={ edge_index=[2, 430342] },
  (implicit_positive, feedback_to_item_implicit_positive, item)={ edge_index=[2, 430342] },
  (implicit_positive, to_user_implicit_positive, user)={ edge_index

In [26]:
data['item'].node_id

tensor([    0,     1,     2,  ..., 25454, 25455, 25456])

In [27]:
def prepare_thp_data(df: pd.DataFrame, max_len: int, pad: int, cls_id: int):
    """
    Build sequences of item ids, event types and timestamps per user for THP training.

    Parameters:
    -----------
    df : DataFrame with columns ['user_id','item_id','event','date']
    max_len : int, maximum sequence length (pad or truncate to this length)
    pad : int, padding token value (left-padding)

    Returns:
    --------
    seq_ids   : LongTensor [num_users, max_len]
    event_type: LongTensor [num_users, max_len]
    seq_times : FloatTensor [num_users, max_len]
    seq_mask  : BoolTensor [num_users, max_len]
    """
    users = df['user_id'].unique()
    num_users = len(users)

    # +1 for the [CLS] token
    new_max_len = max_len + 1
    
    seq_ids    = torch.full((num_users, new_max_len), pad, dtype=torch.long)
    event_type = torch.full((num_users, new_max_len), pad, dtype=torch.long)
    seq_times  = torch.zeros((num_users, new_max_len), dtype=torch.float)
    seq_mask   = torch.zeros((num_users, new_max_len), dtype=torch.bool)

    # map event labels to ints
    label2idx = {label: idx for idx, label in enumerate(df['event'].unique())}

    # устанавливаем CLS-токен в позицию 0
    seq_ids[:, 0]  = cls_id
    event_type[:,0] = cls_id   
    seq_mask[:, 0] = True

    for i, u in enumerate(users):
        user_df = df[df['user_id'] == u].sort_values('date')
        items = user_df['item_id'].values
        types = user_df['event'].map(label2idx).values
        times = pd.to_datetime(user_df['date'], utc=True).values.astype('datetime64[ns]').astype(np.int64) / 1e9
        
        seq = len(items)
        if seq == 0:
            continue

        # вставляем реальные события **cдвинутые на 1** вправо из-за CLS,
        # чтобы первые new_max_len-lengt...new_max_len-1 оказались данными
        length = min(seq, max_len)
        start = max(0, new_max_len - length)
        seq_ids[i, start:]    = torch.tensor(items[-length:],    dtype=torch.long)
        event_type[i, start:] = torch.tensor(types[-length:],    dtype=torch.long)
        seq_times[i, start:]  = torch.tensor(times[-length:],    dtype=torch.float)
        seq_mask[i, start:]   = True

    return seq_ids, event_type, seq_times, seq_mask


In [28]:
PAD_ID = hyperparameters['pad_id'] 
CLS_ID = data['item'].node_id.shape[0]  
hyperparameters['cls_id'] = CLS_ID
max_len = hyperparameters['max_len_of_thp_history']

seq_ids, event_type, seq_times, seq_mask = prepare_thp_data(train, 
                                                            max_len=max_len, 
                                                            pad=PAD_ID,
                                                            cls_id=CLS_ID)
seq_ids[0], event_type[0], seq_times[0], seq_mask[0]

(tensor([25457,  9001,  8173,  4688, 10179,  8313,  7560,  7999, 10156,  9017,
         10317,  9527, 10657, 10465,  9430, 10118,  7847,  8615,  8499, 10624,
          8549,  6047,  2227,  3964,  8672,  6956, 10461,  4306,  9188,  7244,
          9085,  8347, 10637, 10044,  9356, 11191,  4878,  3952,  2054,  4613,
          5456,  6701,  6336,  5633,  9084, 10119,  5372, 10850, 11443,  3245,
          5680,  6149,  4990, 10387,  5511,  9814, 12165,  6265,  7985,  8194,
         12013,  9450,  4163,  5100,  6254,  8927, 10016, 11673, 10982,  7593,
         12154, 13183, 13064, 13201, 13101, 11279, 12593,  4304, 12793, 13370,
          7890, 11610, 13748, 12131, 10526, 11708, 10737,  8000, 14611, 14119,
         12326, 11518, 14413, 12696,   137,  1451,  2228,  4871,   448,   426,
          8821]),
 tensor([25457,     1,     0,     1,     0,     1,     1,     1,     0,     0,
             0,     1,     1,     1,     1,     1,     1,     0,     0,     1,
             1,     1,     0,     

In [29]:
class THPEncoder(nn.Module):
    """
    Multi-head Transformer Hawkes-inspired encoder with local window.
    Integrates exponential decay kernel within last `window_size` events.
    """
    def __init__(self, d_model: int, n_head: int, window_size: int = 50, 
                 decay: float = 1.0, dropout: float = 0.1, max_len: int = 101):
        super().__init__()

        self.max_len = max_len
        # Learnable positional embeddings
        self.pos_emb = nn.Embedding(max_len, d_model)
        # Temporal (time) embedding: simple linear projection from scalar to d_model
        self.time_emb = nn.Linear(1, d_model)
        
        self.heads = nn.ModuleList([
            _THPHead(d_model, decay, window_size, dropout) for _ in range(n_head)
        ])

        self.ffn = nn.Sequential(
                nn.LayerNorm(d_model),
                nn.Linear(d_model, d_model * 4),
                nn.ReLU(),
                nn.Linear(d_model * 4, d_model),
                nn.Dropout(dropout)
            )
        self.final_norm = nn.LayerNorm(d_model)

    def forward(self, emb: torch.Tensor, times: torch.Tensor, mask: torch.BoolTensor = None):
        # emb: [B, L, D], times: [B, L], mask: [B, L]
        B, L, D = emb.shape
        
        positions = torch.arange(L, device=emb.device).unsqueeze(0).expand(B, -1)  # [B, L]
        pe = self.pos_emb(positions)  # [B, L, D]
        te = self.time_emb(times.unsqueeze(-1))  # [B, L, D]
        x = emb + pe + te
        
        attn_out = torch.stack([head(x, times, mask) for head in self.heads], dim=0).sum(0)
        
        # Residual connection + normalization
        x = x + attn_out
        x = x + self.ffn(x)
        
        return self.final_norm(x)  # [B, L, D]

class _THPHead(nn.Module):
    def __init__(self, d_model: int, decay: float, window_size: int, dropout: float,
                pos_lambda: float = None):
        super().__init__()
        self.linear_v = nn.Linear(d_model, d_model, bias=False)
        nn.init.xavier_uniform_(self.linear_v.weight)
        self.temperature = d_model ** 0.5
        self.decay = decay
        self.window_size = window_size
        self.dropout = nn.Dropout(dropout)
        self.input_norm = nn.LayerNorm(d_model)
        self.pos_lambda = pos_lambda or (1.0 / window_size)

    def forward(self, emb: torch.Tensor, times: torch.Tensor, mask: torch.BoolTensor = None):
        B, L, D = emb.size()
        emb_norm = self.input_norm(emb)
        q = emb_norm / self.temperature           # [B, L, D]
        k = emb_norm                              # [B, L, D]
        v = F.elu(self.linear_v(emb_norm))        # [B, L, D]

        if not torch.isfinite(q).all():
            print("NaN/Inf в q:", torch.isnan(q).sum().item(), torch.isinf(q).sum().item())
        if not torch.isfinite(k).all():
            print("NaN/Inf в k:", torch.isnan(k).sum().item(), torch.isinf(k).sum().item())
        if not torch.isfinite(v).all():
            print("NaN/Inf в v:", torch.isnan(v).sum().item(), torch.isinf(v).sum().item())

        # 3) Build pad mask only
        if mask is not None:
            pad_mask = ~mask.unsqueeze(1).expand(-1, L, -1)  # [B, L, L]
        else:
            pad_mask = torch.zeros((B, L, L), dtype=torch.bool, device=emb.device)

        # Always allow self-attention for pad_mask diagonal
        idx = torch.arange(L, device=emb.device)
        pad_mask[:, idx, idx] = False

        scores = torch.bmm(q, k.transpose(1, 2))  # [B, L, L]

        # Apply temporal decay kernel
        delta = (times.unsqueeze(-1) - times.unsqueeze(-2)).clamp(min=0)
        scores = scores * torch.exp(-self.decay * delta)

        # Apply smooth positional decay
        dist = (idx.unsqueeze(0) - idx.unsqueeze(1)).abs().float()  # [L, L]
        pos_decay = torch.exp(-self.pos_lambda * dist).unsqueeze(0)    # [1, L, L]
        scores = scores * pos_decay

        scores = torch.clamp(scores, min=-1e3, max=1e3)
        scores = scores.masked_fill(pad_mask, float('-inf'))

        # Debug range
        finite = scores[~pad_mask]
        # if finite.numel() > 0:
        #     print(f"Диапазон scores до softmax: min={finite.min().item():.3e}, max={finite.max().item():.3e}")

        attn = F.softmax(scores, dim=-1)
        
        if not torch.isfinite(attn).all():
            print("NaN/Inf в attn после softmax:", torch.isnan(attn).sum().item(), torch.isinf(attn).sum().item())
        
        attn = torch.nan_to_num(attn, nan=0.0, posinf=0.0, neginf=0.0)
        attn = self.dropout(attn)

        out = torch.bmm(attn, v)

        return out

In [30]:
class HeteroGNN(nn.Module):
    def __init__(self,
                 num_users: int,
                 num_items: int,
                 feedback_types: list,
                 emb_dim: int = 32,
                 hidden_dim: int = 16,
                 dropout: float = 0.2):
        super().__init__()
        self.feedback_types = feedback_types
        # Embeddings
        self.user_emb = nn.Embedding(num_users, emb_dim)
        # 0 - padding, все остальное - item_id
        self.item_emb = nn.Embedding(num_items + 1, emb_dim, padding_idx=0)  
        self.fb_emb = nn.ModuleDict({ft: nn.Embedding(num_users, emb_dim)
                                     for ft in feedback_types})
        # LayerNorms
        types = ['user', 'item'] + feedback_types
        self.norm1 = nn.ModuleDict({t: nn.LayerNorm(hidden_dim) for t in types})
        self.norm2 = nn.ModuleDict({t: nn.LayerNorm(emb_dim) for t in types})
        # Convolutions
        conv1, conv2 = {}, {}
        for ft in feedback_types:
            conv1[('item', f'to_feedback_{ft}', ft)] = SAGEConv(emb_dim, hidden_dim)
            conv1[(ft, f'feedback_to_item_{ft}', 'item')] = SAGEConv(emb_dim, hidden_dim)
            conv1[(ft, f'to_user_{ft}', 'user')] = SAGEConv(emb_dim, hidden_dim)
            conv2[('item', f'to_feedback_{ft}', ft)] = SAGEConv(hidden_dim, emb_dim)
            conv2[(ft, f'feedback_to_item_{ft}', 'item')] = SAGEConv(hidden_dim, emb_dim)
            conv2[(ft, f'to_user_{ft}', 'user')] = SAGEConv(hidden_dim, emb_dim)
        # user-user
        conv1[('user', 'interacts', 'user')] = SAGEConv(emb_dim, hidden_dim)
        conv2[('user', 'interacts', 'user')] = SAGEConv(hidden_dim, emb_dim)
        self.conv1 = HeteroConv(conv1, aggr='mean')
        self.conv2 = HeteroConv(conv2, aggr='mean')
        self.dropout = nn.Dropout(dropout)

    def forward(self, data):
        # Node features
        x = {
            'user': self.user_emb(data['user'].node_id),
            'item': self.item_emb(data['item'].node_id)
        }
        for ft in self.feedback_types:
            x[ft] = self.fb_emb[ft](data[ft].node_id)
            
        h1 = self.conv1(x, data.edge_index_dict)
        h1 = {t: self.dropout(F.leaky_relu(self.norm1[t](h1[t]))) for t in h1}
        
        h2 = self.conv2(h1, data.edge_index_dict)
        out = {t: self.norm2[t](h2[t]) for t in h2}
        return out['user']

In [31]:
class Model(nn.Module):
    def __init__(self,
                 num_users: int,
                 num_items: int,
                 feedback_types: list,
                 d_model: int = 32,
                 n_head: int = 4,
                 window_size: int = 50,
                 decay: float = 1.0,
                 dropout: float = 0.1):
        super().__init__()
        # Static graph encoder
        self.gnn = HeteroGNN(num_users, num_items, feedback_types,
                             emb_dim=d_model, hidden_dim=d_model//2,
                             dropout=dropout)
        # Inlined THP sequence encoder
        self.thp = THPEncoder(d_model=d_model,
                              n_head=n_head,
                              window_size=window_size,
                              decay=decay,
                              dropout=dropout)

    def forward(self, data, seq_ids, seq_times, seq_mask, batch_users):
        # Static graph embeddings
        user_embs = self.gnn(data)          # [num_users, d_model]
        # Sequence encoding
        seq_item_emb = self.gnn.item_emb(seq_ids)  # [B, L, d_model]
        attn_out = self.thp(seq_item_emb, seq_times, seq_mask)
        seq_rep = attn_out[:, -1, :]        # [B, d_model]
        # Get static user embeddings
        gnn_rep = user_embs[batch_users]   # [B, d_model]
        # Updated user embedding
        updated_user_emb = seq_rep + gnn_rep
        return updated_user_emb

In [32]:
num_users = data['user'].node_id.shape[0]      
num_items = data['item'].node_id.shape[0]      
feedback_types = train['target'].unique().tolist()
data.user_idx = data['user'].node_id
d_model = hyperparameters['thp_dmodel']             
n_head = hyperparameters['thp_n_head']          
window_size = hyperparameters['thp_window_size']     
decay = hyperparameters['thp_decay']         
dropout = hyperparameters['thp_dropout']           

model = Model(
    num_users=num_users,
    num_items=num_items,
    feedback_types=feedback_types,
    d_model=d_model,
    n_head=n_head,
    window_size=window_size,
    decay=decay,
    dropout=dropout
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [33]:
num_items

25457

In [34]:
B = 32
seq_ids_batch   = seq_ids[:B]     # [B, L]
seq_times_batch = seq_times[:B]   # [B, L]
seq_mask_batch  = seq_mask[:B]    # [B, L]

item_emb = model.gnn.item_emb 
d_model = item_emb.embedding_dim

# Получаем seq_item_emb: [B, L, D]
seq_item_emb = item_emb(seq_ids_batch)

thp_encoder = THPEncoder(
    d_model=d_model,
    n_head=4,
    window_size=50,
    decay=1.0,
    dropout=0.1
)

thp_encoder.to(device)
seq_item_emb   = seq_item_emb.to(device)
seq_times_batch= seq_times_batch.to(device)
seq_mask_batch = seq_mask_batch.to(device)

out = thp_encoder(
    emb=seq_item_emb,
    times=seq_times_batch,
    mask=seq_mask_batch
)

print("THPEncoder output shape:", out.shape)  # ожидаем [B, L, D]

THPEncoder output shape: torch.Size([32, 101, 64])


In [35]:
B = 32
batch_seq_ids   = seq_ids[:B].to(device)    # [B, L]
batch_seq_times = seq_times[:B].to(device)  # [B, L]
batch_seq_mask  = seq_mask[:B].to(device)   # [B, L]

# data.user_idx = data['user'].node_id[:B]
batch_users = data.user_idx[:B].to(device)
model.to(device)
data.to(device)

updated_user_emb = model(
    data=data,
    seq_ids=batch_seq_ids,
    seq_times=batch_seq_times,
    seq_mask=batch_seq_mask,
    batch_users=batch_users
)  # [B, d_model]

print("Updated user embeddings:", updated_user_emb.shape)

Updated user embeddings: torch.Size([32, 64])


In [36]:
train.item_id.nunique(), train.item_id.min(), train.item_id.max()

(25456, 1, 25456)

In [37]:
model

Model(
  (gnn): HeteroGNN(
    (user_emb): Embedding(18892, 64)
    (item_emb): Embedding(25458, 64, padding_idx=0)
    (fb_emb): ModuleDict(
      (implicit_negative): Embedding(18892, 64)
      (explicit_positive): Embedding(18892, 64)
      (implicit_positive): Embedding(18892, 64)
      (expliсit_negative): Embedding(18892, 64)
    )
    (norm1): ModuleDict(
      (user): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (item): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (implicit_negative): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (explicit_positive): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (implicit_positive): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (expliсit_negative): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
    (norm2): ModuleDict(
      (user): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (item): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (implicit_

In [38]:
test_df = test[['user_id', 'item_id']]
interactions = test_df.rename(columns={
    'user_id': Columns.User,
    'item_id': Columns.Item,
})

viewed_items = train.groupby("user_id")["item_id"].agg(set).to_dict()

In [39]:
CLS_ID == model.gnn.item_emb.weight.shape[0] - 1

True

In [40]:
def evaluate(model, train_data, seq_train_data,
             test_batch_size, top_k,
             viewed_items, interactions,
             device, test_step):
    """
    Оцениваем модель по всем пользователям:
    - строим топ-K рекомендации
    - фильтруем уже просмотренные
    - считаем recall@K, precision@K, map@K
    """
    model.eval()
    model.to(device)
    seq_ids, event_type, seq_times, seq_mask = seq_train_data
    num_users = seq_ids.size(0)
    test_top_k = top_k * 150

    item_emb = model.gnn.item_emb.weight
    num_items = item_emb.shape[0]
    item_emb_t = item_emb.t().detach()
    del item_emb
    gc.collect()

    all_scores = []
    with torch.no_grad():
        for i in range(0, num_users, test_batch_size):
            end = min(i + test_batch_size, num_users)
            batch_users = torch.arange(i, end).to(device)
            s_ids   = seq_ids[i:end].to(device)
            s_times = seq_times[i:end].to(device)
            s_mask  = seq_mask[i:end].to(device)
            user_e = model(
                data=train_data.to(device),
                seq_ids=s_ids,
                seq_times=s_times,
                seq_mask=s_mask,
                batch_users=batch_users
            )
            rating = torch.mm(user_e.detach(), item_emb_t)
            _, topk = torch.topk(rating, k=test_top_k, dim=1)
            all_scores.append(topk)

            del user_e, rating
            gc.collect()
    all_scores = torch.cat(all_scores, dim=0).cpu().numpy()

    users_list, items, ranks = [], [], []
    for u in range(num_users):
        seen = viewed_items.get(u, set())
        recs = all_scores[u]
        mask = (
            (~np.isin(recs, list(seen)))   
            & (recs != 0)                  
            & (recs != num_items - 1)     
            )
        filtered = recs[mask][:top_k]
        for rank, it in enumerate(filtered, 1):
            users_list.append(u)
            items.append(int(it))
            ranks.append(rank)
    reco_df = pd.DataFrame({
        'user_id': users_list,
        'item_id': items,
        'rank': ranks
    })

    metrics = {
        f'map@{top_k}': MAP(k=top_k),
        f'precision@{top_k}': Precision(k=top_k),
        f'recall@{top_k}': Recall(k=top_k),
        f'ndcg@{top_k}': NDCG(k=top_k)
    }
    results = calc_metrics(metrics=metrics,
                           reco=reco_df,
                           interactions=interactions)
    print(f"Step {test_step} — Test metrics:")
    for name, val in results.items():
        print(f"  {name}: {val:.9f}")
        experiment.log_metric(f"Test {name} vs step", val, step=test_step)
    del all_scores
    gc.collect()

    model.to(device)
    train_data.to(device)
    model.train()
    return results

In [41]:
def train_model(model: HeteroGNN,
                train_data: HeteroData,
                seq_train_data: tuple,
                edge_type: tuple,
                num_epochs: int = 10,
                lr: float = 1e-3,
                batch_size: int = 1024,
                device: str = None,
                print_every: int = 100,
                test_every: int = 500,
                top_k: int = 10,
                test_batch_size=2048,
                scheduler_step_size: int = 1,
                scheduler_gamma: float = 0.9) -> Model:
    seq_ids, event_type, seq_times, seq_mask = seq_train_data
    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    train_data = train_data.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=scheduler_step_size, gamma=scheduler_gamma)

    if isinstance(edge_type, list):
        src_list, dst_list = [], []
        for et in edge_type:
            s, d = train_data[et].edge_index
            src_list.append(s)
            dst_list.append(d)
        src = torch.cat(src_list, dim=0)
        dst = torch.cat(dst_list, dim=0)
    else:
        src, dst = train_data[edge_type].edge_index
    
    num_train = src.size(0)
    test_top_k = top_k * 150
    total_steps = 0
    
    print(f"Num of training examples: {num_train}")
    for epoch in range(1, num_epochs + 1):
        model.train()
        perm = torch.randperm(num_train, device=device)
        total_loss = 0.0
        running_loss = 0.0
        running_steps = 0
        step = 0

        for i in range(0, num_train, batch_size):
            idx = perm[i:i + batch_size]
            users = dst[idx]
            cpu_users = users.to('cpu')

            seq_ids_batch = seq_ids[cpu_users].to(device)
            seq_times_batch = seq_times[cpu_users].to(device)
            seq_mask_batch = seq_mask[cpu_users].to(device)
            
            pos_items = src[idx]
            neg_items = torch.randint(1, model.gnn.item_emb.num_embeddings - 1,
                                      size=pos_items.size(), device=device)
            
            optimizer.zero_grad()
            user_embs = model(data=train_data, 
                              seq_ids=seq_ids_batch,
                              seq_times=seq_times_batch,
                              seq_mask=seq_mask_batch,
                              batch_users=users)
            
            pos_emb = model.gnn.item_emb(pos_items)
            neg_emb = model.gnn.item_emb(neg_items)
            pos_score = (user_embs * pos_emb).sum(dim=1)
            neg_score = (user_embs * neg_emb).sum(dim=1)
            diff = pos_score - neg_score
            diff = torch.clamp(diff, -10.0, 10.0)
            loss = -torch.log(torch.sigmoid(diff) + 1e-15).mean()
            
            nan_mask = torch.isnan(diff)            
            if nan_mask.any():
                idxs = torch.nonzero(nan_mask).squeeze()
                print(f"!!! FOUND {nan_mask.sum().item()} NaN(s) in diff at positions: {idxs.tolist()}")

            # with torch.autograd.detect_anomaly():
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()
            running_loss += loss.item()
            running_steps += 1
            step += 1

            experiment.log_metric('Train Loss vs step', loss.item(), step=total_steps)
            
            if step % print_every == 0 or step == 1:
                avg_loss = running_loss / running_steps
                current_lr = optimizer.param_groups[0]['lr']
                d = diff.detach().cpu()
                print(f"Epoch {epoch}, Step {step}, LR: {current_lr:.6f}, Current Loss: {loss.item():.4f}, Avg Loss: {avg_loss:.4f}")
                print(f"Diff stats — min: {d.min():.4f}, max: {d.max():.4f}, mean: {d.mean():.4f}, std: {d.std():.4f}")
                print()

                experiment.log_metric('Diff stats (mean) vs step', d.mean(), step=total_steps)
                experiment.log_metric('Diff stats (std) vs step', d.std(), step=total_steps)

            del user_embs, pos_emb, neg_emb, pos_score, neg_score,\
            seq_ids_batch, seq_times_batch, seq_mask_batch
            gc.collect()
            torch.cuda.empty_cache()

            scheduler.step()
            
            if step % test_every == 0 or step == 1:
                evaluate(model, train_data, seq_train_data,
                         test_batch_size, top_k,
                         viewed_items, interactions,
                         device, test_step=total_steps)
            total_steps += 1
        epoch_loss = total_loss / num_train
        experiment.log_metric(f'Train Loss vs epoch', epoch_loss, epoch=epoch)
        print(f"Epoch {epoch} completed, Train Loss: {epoch_loss:.6f}")
        print()
    return model

In [42]:
experiment.log_parameters(hyperparameters)

In [43]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [44]:
model = Model(
    num_users=num_users,
    num_items=num_items,
    feedback_types=feedback_types,
    d_model=d_model,
    n_head=n_head,
    window_size=window_size,
    decay=decay,
    dropout=dropout
)

edge_type = hyperparameters['train_edge_type']
num_epochs = hyperparameters['train_num_epochs']
lr = hyperparameters['train_lr']
batch_size = hyperparameters['train_batch_size']
print_every = hyperparameters['train_print_every']
test_every = hyperparameters['train_test_every']
top_k = hyperparameters['test_topk']
test_batch_size = hyperparameters['test_batch_size']
scheduler_step_size = hyperparameters['train_scheduler_step_size']
train_scheduler_gamma = hyperparameters['train_scheduler_gamma']

model = train_model(model,
                    data,
                    (seq_ids, event_type, seq_times, seq_mask),
                    edge_type=edge_type,
                    num_epochs=num_epochs,
                    lr=lr,
                    batch_size=batch_size,
                    print_every=print_every,
                    test_every=test_every,
                    top_k=top_k,
                    test_batch_size=test_batch_size,
                    scheduler_step_size=scheduler_step_size,
                    scheduler_gamma=train_scheduler_gamma)

Num of training examples: 789805
Epoch 1, Step 1, LR: 0.001000, Current Loss: 3.7312, Avg Loss: 3.7312
Diff stats — min: -10.0000, max: 10.0000, mean: 0.0042, std: 8.0775

Step 0 — Test metrics:
  precision@10: 0.000508721
  recall@10: 0.000510923
  ndcg@10: 0.000502022
  map@10: 0.000144033
Epoch 1, Step 20, LR: 0.001000, Current Loss: 3.0158, Avg Loss: 3.4238
Diff stats — min: -10.0000, max: 10.0000, mean: -0.0352, std: 6.7506

Epoch 1, Step 40, LR: 0.001000, Current Loss: 1.9504, Avg Loss: 2.8849
Diff stats — min: -10.0000, max: 10.0000, mean: -0.0518, std: 4.4267

Step 49 — Test metrics:
  precision@10: 0.000336945
  recall@10: 0.000337679
  ndcg@10: 0.000314029
  map@10: 0.000085259
Epoch 1, Step 60, LR: 0.001000, Current Loss: 1.3061, Avg Loss: 2.4324
Diff stats — min: -10.0000, max: 9.6958, mean: 0.0591, std: 2.8453

Epoch 1, Step 80, LR: 0.001000, Current Loss: 1.0685, Avg Loss: 2.1215
Diff stats — min: -6.9760, max: 7.6592, mean: 0.0973, std: 2.1675

Epoch 1, Step 100, LR: 0.0

KeyboardInterrupt: 

In [45]:
torch.save(model, "gnn_model_mvl.model")
from IPython.display import FileLink

FileLink('gnn_model_mvl.model')

/kaggle/working/gnn_model_mvl.model

In [46]:
# del model
gc.collect()
torch.cuda.empty_cache()

In [47]:
log_model(
    experiment=experiment,
    model=model,
    model_name="GNN+THP",
)

In [48]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : emoSAGE+THP-books
COMET INFO:     url                   : https://www.comet.com/annanet/gnn-recommender/5321f7ecb6044c33a138b28b232b1381
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Diff stats (mean) vs step [683] : (-0.051830172538757324, 4.259060382843018)
COMET INFO:     Diff stats (std) vs step [683]  : (0.6834704875946045, 8.077468872070312)
COMET INFO:     Test map@10 vs step [273]       : (8.525873351454747e-05, 0.003668978277142037)
COMET INFO:     Test ndcg@10 vs step [273]      : (0.0003140293936996998, 0.011515632986891637)
COMET INFO:     Test precision@10 vs step [273] : (0.00033694503171247353, 0.011218287526427062)
COMET I